# Prediction with .h5 and .tflite file

### Imports


In [1]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import os
import pandas as pd
import numpy as np

import tools

# Root CSV files directory
#dirname = "./prediction_data/"
dirname = "./data/absolute/2D/"

# Frame count
frames = 100

In [3]:
model = tf.keras.models.load_model('model-best.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 64)           38656     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 128)          98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 12)                1548      
Total params: 270,604
Trainable params: 270,604
Non-trainable params: 0
_________________________________________________________________


#### Load the file to predict and tokenize labels for comparison

In [5]:
listfile = os.listdir(dirname)
data = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True) 
        data.append((wordname, content))
features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
labels = [n[0] for n in data]
x_train=np.array(features, dtype="float32")
y_train = np.array(labels, dtype="float32")
print(labels)
tokenizer = tools.tokenize(dirname)
token_labels = {y:x for x,y in tokenizer.word_index.items()}
print(token_labels)


ValueError: could not convert string to float: 'unser'

In [4]:

y_pred = model.predict(x_train)


In [5]:
predictions = np.array([np.argmax(pred) for pred in y_pred])
print(predictions)
print([token_labels[p] for p in predictions])

[4 4 4 4 3 4 4 3 4 4 4 2 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 3 4
 4 4 4 4 4 1 4 1 4 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4
 4 4 4 4 3 4 4 4 2 4 4 3 1 4 4 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 3 2 2 2 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 2 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 3 3 3 3 3 3 2 3 3 3 3 3 3 3
 3 3 1 3 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1
 4 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1
 1 1 1 1 1 4 1 1 1 1 1 1 1 1]
['welt', 'welt', 'welt', 'welt', 'hallo', 'welt', 'welt', 'hallo', 'welt', 'welt', 'welt', 'deutschland', 'welt', 'computer', 'welt', 'welt', 'welt', 'welt', 'welt', 'welt', 'welt', 'welt', 'welt', 'welt', 'welt', 'welt', 'welt', 'hallo', 'welt', 'welt', 'welt', 'welt', 'welt', 'welt', 'welt', 

https://www.tensorflow.org/lite/guide/inference#load_and_run_a_model_in_python

In [7]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="sign_lang_recognition.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
# Test model on random input data.

#interpreter.set_tensor(0.0, x_train)

input_shape = input_details[0]['shape']

input_data = [1]
predictions = []
i= 1
for x_pred in x_train:
    input_data[0] = x_pred

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()    
# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predictions.append(np.argmax(output_data))
    print("prediction:",token_labels[np.argmax(output_data)], "wahrheit: ", labels[i] )
    i = i+ 1
predictions = [token_labels[p] for p in predictions]
print("Predictions:", predictions)

[{'name': 'lstm_input', 'index': 0, 'shape': array([  1, 100,  86]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: hallo wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: computer wahrheit:  Computer
prediction: 

prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: computer wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: computer wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: computer wahrheit:  Hallo
prediction: computer wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
prediction: hallo wahrheit:  Hallo
predicti

IndexError: list index out of range